# Multilingual RAG + LLM Integration with Gemini API

## 1. Setup and Installation

In [31]:
# Install required packages
!pip install google-generativeai langchain langchain-google-genai chromadb sentence-transformers python-dotenv langdetect

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993332 sha256=a18dc31cd16914e5797dfbb45e6d9585cd5967f3664efbc7e84ecd64f20a6718
  Stored in directory: /home/duyle/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [32]:
import os
import json
import pickle
from typing import List, Dict, Any
from pathlib import Path
import langdetect
# RAG components
import chromadb
from sentence_transformers import SentenceTransformer

# Langchain components
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

# Environment variables
from dotenv import load_dotenv
load_dotenv('/home/duyle/Documents/Case-Study/.env')

True

## 2. API Key Setup

In [19]:
load_dotenv('/home/duyle/Documents/Case-Study/.env')

True

In [20]:
# Set up your Gemini API key
# Option 1: Set in environment variable
# os.environ["GOOGLE_API_KEY"] = "your-api-key-here"

# Option 2: Load from .env file (recommended)
# Create a .env file with: GOOGLE_API_KEY=your-api-key-here
api_key = os.getenv("GOOGLE-API-KEY")


## 3. Load Existing RAG Data

In [21]:
with open('data/processed/chunked_documents.pkl', 'rb') as f:
    chunked_documents = pickle.load(f)


## 4. Initialize Vector Store

In [ ]:
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")


In [ ]:
# Langchain wrapper 
embeddings = SentenceTransformerEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={})

In [ ]:
persist_directory = "./chroma_db_langchain_e5"

if os.path.exists(persist_directory) and os.listdir(persist_directory):
    vectorstore = Chroma( persist_directory=persist_directory, embedding_function=embeddings)
else:

    texts = [doc.page_content for doc in chunked_documents]
    metadatas = [doc.metadata for doc in chunked_documents]
    # Add task prefixes
    texts_with_prefix = [f"passage: {text}" for text in texts]
    
    vectorstore = Chroma.from_texts(
        texts=texts_with_prefix,
        metadatas=metadatas,
        embedding=embeddings,
        persist_directory=persist_directory
    )



✅ Created vector store with 257 documents


## 5. Initialize Gemini LLM

In [25]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1,
    convert_system_message_to_human=True,
    google_api_key = api_key)

print("✅ Gemini LLM initialized")

✅ Gemini LLM initialized


## 6. Create Custom RAG Chain

In [ ]:
def detect_language(text):
    """Enhanced language detection for mixed Vietnamese-English queries"""
    
    from langdetect import detect
    detected = detect(text)
    
    vietnamese_chars = set('àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ')
    has_vietnamese = any(char in vietnamese_chars for char in text.lower())
    
    if has_vietnamese or detected == 'vi':
        return 'vi'
    else:
        return 'en'


In [ ]:

def get_language_specific_prompt(language):
    """Get prompt template based on language"""
    if language == 'vi':
        vietnamese_template = """Bạn là trợ lý AI chuyên về APEC 2025 Korea.

        Hướng dẫn:
        - Sử dụng thông tin được cung cấp để trả lời câu hỏi một cách chính xác
        - Khi trình bày dữ liệu bảng, định dạng rõ ràng và dễ đọc
        - Bao gồm các chi tiết cụ thể như ngày tháng, địa điểm và tên sự kiện
        - Nếu ngữ cảnh chứa nhiều mục liên quan, hãy liệt kê tất cả
        - Nếu không tìm thấy câu trả lời trong ngữ cảnh, hãy nói rõ ràng
        - Trả lời bằng tiếng Việt

        Ngữ cảnh:
        {context}

        Câu hỏi: {question}

        Trả lời:"""
        
        return PromptTemplate(
            template=vietnamese_template,
            input_variables=["context", "question"]
        )
    else:
        english_template = """You are an expert AI assistant specializing in APEC 2025 Korea information.

        Instructions:
        - Use the provided context to answer questions accurately
        - When presenting table data, format it clearly and readably
        - Include specific details like dates, venues, and event names
        - If the context contains multiple relevant items, list them all
        - If you cannot find the answer in the context, say so clearly
        - Maintain accuracy of all factual information

        Context:
        {context}

        Question: {question}

        Answer:"""
        
        return PromptTemplate(
            template=english_template,
            input_variables=["context", "question"]
        )



In [37]:
# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 5, 
        }
    ),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

print("✅ RAG chain created successfully")

✅ RAG chain created successfully


## 7. Enhanced RAG Functions

In [ ]:
def enhanced_rag_query(question, show_sources= True):
    
    # 1. Detect language 
    detected_language = detect_language(question)
    
    # 2. Get language-specific prompt
    language_prompt = get_language_specific_prompt(detected_language)
    
    # 3. Create temporary chain with language-specific prompt
    temp_qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        ),
        chain_type_kwargs={"prompt": language_prompt},
        return_source_documents=True
    )
    
    search_question = f"query: {question}"
    
    result = temp_qa_chain({"query": search_question})
    
    sources = []
    for doc in result["source_documents"]:
        source_info = {
            "title": doc.metadata.get("title", "Unknown"),
            "url": doc.metadata.get("url", ""),
            "contains_table": doc.metadata.get("contains_table", False),
            "chunk_length": doc.metadata.get("chunk_length", 0)
        }
        sources.append(source_info)
    
    response = {
        "answer": result["result"],
        "sources": sources,
        "num_sources": len(sources),
        "detected_language": detected_language
    }
    
    if show_sources:
        lang_flag = "vi" if detected_language == 'vi' else "en"
        print(f"{lang_flag} Answer based on {len(sources)} sources:")
        print(f"Tables included: {sum(1 for s in sources if s['contains_table'])}")
        print("\n" + "="*60)
        print(response["answer"])
        print("\n" + "="*60)
        print("Sources:")
        for i, source in enumerate(sources, 1):
            table_indicator = "Table used" if source["contains_table"] else "No table used"
            print(f"{i}. {table_indicator} {source['title']}")
    
    return response

## 8. Test the RAG System

In [ ]:
# Test 1: General question
print("Test 1: General APEC question")
response1 = enhanced_rag_query("APEC là gì")

Test 1: General APEC question


/home/duyle/.pyenv/versions/3.10.16/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


vi Answer based on 5 sources:
Tables included: 0

APEC (Asia-Pacific Economic Cooperation) là một diễn đàn kinh tế khu vực được thành lập vào năm 1989 để tận dụng sự phụ thuộc lẫn nhau ngày càng tăng của khu vực Châu Á - Thái Bình Dương. APEC có 21 thành viên, với mục tiêu tạo ra sự thịnh vượng lớn hơn cho người dân trong khu vực bằng cách thúc đẩy tăng trưởng cân bằng, toàn diện, bền vững, đổi mới và an toàn, đồng thời đẩy nhanh hội nhập kinh tế khu vực. APEC đảm bảo hàng hóa, dịch vụ, đầu tư và con người di chuyển trơn tru qua biên giới. Các thành viên tạo điều kiện thuận lợi cho thương mại thông qua việc hợp lý hóa các thủ tục hải quan tại biên giới; thúc đẩy môi trường kinh doanh thuận lợi hơn; và điều phối các quy định và tiêu chuẩn trong khu vực. Đây là diễn đàn liên chính phủ toàn cầu duy nhất trên thế giới cam kết giảm các rào cản đối với thương mại và đầu tư mà không có các nghĩa vụ ràng buộc về mặt pháp lý. APEC đạt được các mục tiêu của mình bằng cách thúc đẩy đối thoại và đ

In [ ]:
response1 = enhanced_rag_query("Lịch meetings APEC 2025")

/home/duyle/.pyenv/versions/3.10.16/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


vi Answer based on 5 sources:
Tables included: 3

Dưới đây là lịch các cuộc họp APEC 2025:

| No. | Tên sự kiện                                                                 | Ngày                | Địa điểm           |
|-----|------------------------------------------------------------------------------|---------------------|--------------------|
| 1   | Informal Senior Officials’ Meeting (ISOM)                                  | 9 - 11 tháng 12, 2024 | Seoul              |
| 2   | 1st APEC Business Advisory Council Meeting (ABAC)                           | 23 – 25 tháng 2, 2025 | Brisbane, Australia |
| 3   | First Senior Officials’ Meeting and Related Meetings (SOM1)                  | 24 tháng 2 - 9 tháng 3, 2025 | Gyeongju           |
| 4   | Finance and Central Bank Deputies’ Meeting (FCBDM)                          | 6 - 7 tháng 3, 2025   | Gyeongju           |
| 5   | 2nd APEC Business Advisory Council Meeting (ABAC)                           | 23 - 26 tháng 4, 2025 | Toronto

In [ ]:
print("\n🔍 Test 2: Table-specific question")
response2 = enhanced_rag_query("Cho tôi biết về các cuộc họp APEC được xếp vào tháng 5 2025")


🔍 Test 2: Table-specific question


/home/duyle/.pyenv/versions/3.10.16/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


vi Answer based on 5 sources:
Tables included: 2

Dưới đây là danh sách các cuộc họp APEC diễn ra vào tháng 5 năm 2025:

| No. | Tên sự kiện                                                      | Ngày          | Địa điểm |
|-----|-------------------------------------------------------------------|---------------|---------|
| 7   | Second Senior Officials’ Meeting and Related Meetings (SOM2)       | 3 - 16/5/2025 | Jeju    |
| 8   | Human Resource Development Ministerial Meeting (HRDMM)            | 11 - 13/5/2025| Jeju    |
| 9   | APEC Education Ministerial Meeting(AEMM)                          | 13 - 15/5/2025| Jeju    |
| 10  | Ministers Responsible for Trade (MRT)                             | 15 - 16/5/2025| Jeju    |

Ngoài ra, có một số sự kiện bên lề diễn ra vào tháng 5 năm 2025:

| No. | Tên sự kiện                                            | Ngày       | Địa điểm |
|-----|---------------------------------------------------------|------------|---------|
| 1   | International

In [ ]:
print("\nTest 3: Venue-specific question")
response3 = enhanced_rag_query("Có cuộc họp nào vào tháng 5 at JeJu khong")


Test 3: Venue-specific question


/home/duyle/.pyenv/versions/3.10.16/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


vi Answer based on 5 sources:
Tables included: 1

Có, có nhiều cuộc họp diễn ra tại Jeju vào tháng 5 trong khuôn khổ APEC 2025 Korea. Dưới đây là danh sách các sự kiện được liệt kê:

*   **International Forum on Disability Employment:** 6 tháng 5, 2025
*   **APEC Future Education Forum (AFEF):** 6 tháng 5, 2025
*   **HRDMM : Policy Experience Booths:** 6 - 12 tháng 5, 2025
*   **APEC Sustainable Social Entrepreneurship Training (ASSET):** 10 tháng 5, 2025
*   **Global Education Reform Conference:** 13 tháng 5, 2025
*   **APEC University Leader’s Forum (AULF):** 13 tháng 5, 2025
*   **HRDMM : Field Trip:** 13 tháng 5, 2025
*   **Educational Innovation Achievement Sharing Exhibition Booth:** 13 - 14 tháng 5, 2025
*   **APEC Second Senior Officials’ Meeting (SOM2):** 3 - 16 tháng 5, 2025
*   **School Visit and Field Trip:** 15 tháng 5, 2025
*   **Bilateral talks with trade representatives:** 14-16 tháng 5, 2025

Sources:
1. Table used Side Events > APEC 2025 KOREA
2. No table used Press R